<a href="https://colab.research.google.com/github/seungyhyunMacc/FullDuplex_LearningMachine/blob/main/FullDuplex_LearningModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

\begin{align}
y[n]= \sum_\limits{p=1,p=odd}^{P} \sum\limits_{q=0}^{p} \sum\limits_{l=0}^{L} h_{p,q}[l]s[n-l]^{q}s[n-l]^{p-q}
\end{align}

\begin{align}
y[n]= \sum_\limits{p=1,p=odd}^{P}\sum\limits_{l=0}^{L} h_{p}[l]s[n-l]^{p}
\end{align}

In [28]:
length_data = 100000
L = 3   
var_S = 5e-1
var_N = 1e-5

P_max = 5
P_min = 1

# traning set 
s = np.zeros(length_data, dtype='complex') # the transmitted signal
s.real = np.sqrt(var_S/2)*np.random.randn(length_data) 
s.imag = np.sqrt(var_S/2)*np.random.randn(length_data)

print('s\n')
print(s)  
print('\n')
print(s.conjugate())

y = np.zeros(length_data + L - 1, dtype='complex')  # the received signal 
print('y\n')
print(y)
print('\n')


########################################################
#  Training Data Set
#######################################################
h = np.zeros([P_max+1, P_max+1, L], dtype='complex') # the transmitted signal
h.real = np.random.randn(P_max+1, P_max+1, L) 
h.imag = np.random.randn(P_max+1, P_max+1, L)
print('h\n')
print(h)
print('\n')

# Received symbols 
for p in [a for a in range(P_min, P_max+1) if a%2==1]: # 1 3 5
  for q in range(p+1): # 0~1 0~3 0~5
    middle = np.power(s, q) * np.power(s.conjugate(), p-q)
    y = y + np.convolve(h[p][q], middle)
    print("h[p] = ", p, h[p][q])

print('y\n')
print(y)
print('\n')

noise = np.zeros(length_data + L-1, dtype='complex')
noise.real = np.sqrt(var_N/2)*np.random.randn(length_data + L-1) 
noise.imag = np.sqrt(var_N/2)*np.random.randn(length_data + L-1) 

y = y + noise
print('y\n')
print(y.shape)
print('\n')


#####################################################
# # Z = np.zeros(L+L-1,dtype='float32') 
# for p in [a for a in range(P_min,P_max+1) if a%2==1]:
#   for q in range(p):
#     y = y + np.convolve(h[p],np.power(s,p))

#     Z = Z + np.convolve(h[p],np.power(x_data[33],p))

# # print(Z)
# # print(y_data[33])
# # print(y_data[33]-Z[2])

s

[ 0.34518342+1.34287724j  0.77888422+0.98799969j -0.15627797-0.59743346j
 ... -0.26643489+0.07706517j  0.40415046+0.3125661j
  0.382     +0.20396542j]


[ 0.34518342-1.34287724j  0.77888422-0.98799969j -0.15627797+0.59743346j
 ... -0.26643489-0.07706517j  0.40415046-0.3125661j
  0.382     -0.20396542j]
y

[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]


h

[[[-1.56524283e+00-1.70857899j -5.38809471e-01-0.525042j
   -1.92172226e-01+0.54642711j]
  [-1.45611724e+00+0.28614057j  1.01198750e+00-0.57460435j
    8.56774682e-01-1.04369322j]
  [-2.45300113e-01-1.26487132j -5.96205458e-02+0.0235107j
   -8.67118433e-01-0.65132105j]
  [ 2.43097162e-01-1.10812929j  1.65997432e+00+0.79294046j
    7.96011692e-01+1.23938893j]
  [-6.21617700e-01-1.44155282j  1.36202691e+00-0.76076851j
    7.69509775e-01-1.02452369j]
  [ 6.52928136e-02-0.81446881j  7.95370244e-01-1.05760267j
   -1.30940613e-02+0.38901579j]]

 [[ 6.02158190e-01-1.04475382j  6.43573486e-01+1.46131555j
    2.37199949e-01+0.48973736j]
 

In [3]:
np.convolve([1+2j, 2+3j, 3+2j], [0+1j, 1+1j, 2+3j])

array([-2. +1.j, -4. +5.j, -7.+15.j, -4.+17.j,  0.+13.j])

In [31]:
### polynomial of x_data
## Training data generation 
x_data = np.zeros([length_data - L + 1, L],dtype = 'complex')   ## feature data
y_data = np.zeros([length_data - L + 1, 1],dtype = 'complex')   ## label 

### mapping
for i in range(length_data - L + 1):
    y_data[i] = y[i + L -1] ## y값을 y_data에 매핑 
    x_data[i] = s[i : i + L]  ## s값(송신신호)를 x_data에 매핑

x_data2 = x_data

for p in [a for a in range(P_min,P_max+1) if a%2==1]:
  for q in range(p+1):
    if p == 1 and q == 0:
      x_data = np.power(x_data2, q)*np.power(x_data2.conjugate(), p-q)
    else:
      math_sentence_answer = np.power(x_data2, q)*np.power(x_data2.conjugate(), p-q)
      x_data = np.concatenate((x_data, math_sentence_answer), axis=1)

train_data = np.concatenate((x_data.real, x_data.imag), axis=1)
target_data = np.concatenate((y_data.real, y_data.imag), axis=1)


print('x데이터')
print(train_data.shape)
print('\n')

print('y데이터')
print(target_data.shape)
print('\n')

# x_data_test2 = x_data_test
# for p in [a for a in range(P_min,P_max+1) if a%2==1]:
#   for q in range(p+1):
#     if p == 1 and q == 0:
#       x_data_test = np.power(x_data_test2, q)*np.power(x_data_test2.conjugate(), p-q)
#     else:
#       math_sentence_answer = np.power(x_data_test2, q)*np.power(x_data_test2.conjugate(), p-q)
#       x_data_test = np.concatenate((x_data_test, math_sentence_answer), axis=1)

# print(x_data_test.shape)
# test_data = np.concatenate((x_data_test.real, x_data_test.imag), axis=1)
# print(test_data.shape)

x데이터
(99998, 72)


y데이터
(99998, 2)




In [35]:
##################################################    
# Test Data Set
###################################################
y = np.zeros(length_data + L - 1, dtype='complex')

s = np.zeros(length_data, dtype='complex') # the transmitted signal
s.real = np.sqrt(var_S/2)*np.random.randn(length_data) 
s.imag = np.sqrt(var_S/2)*np.random.randn(length_data)

for p in [a for a in range(P_min, P_max+1) if a%2==1]: # 1 3 5
  for q in range(p+1): # 0~1 0~3 0~5
    middle = np.power(s, q) * np.power(s.conjugate(), p-q)
    y = y + np.convolve(h[p][q], middle)
    print("h[p] = ", p, h[p][q])

# y = y + np.sqrt(var_N)*np.random.randn(length_data + L-1)

noise2 = np.zeros(length_data + L-1, dtype='complex')
noise2.real = np.sqrt(var_N/2)*np.random.randn(length_data + L-1) 
noise2.imag = np.sqrt(var_N/2)*np.random.randn(length_data + L-1) 
y = y + noise2
print(y.shape)

x_data_test = np.zeros([length_data - L+ 1, L],dtype = 'complex')
y_data_test = np.zeros([length_data - L+ 1, 1],dtype = 'complex')
    
for i in range(length_data- L + 1):
    y_data_test[i] = y[i + L -1]
    x_data_test[i] = s[i : i + L]   

x_data_test2 = x_data_test

for p in [a for a in range(P_min,P_max+1) if a%2==1]:
  for q in range(p+1):
    if p == 1 and q == 0:
      x_data_test = np.power(x_data_test2, q)*np.power(x_data_test2.conjugate(), p-q)
    else:
      math_sentence_answer = np.power(x_data_test2, q)*np.power(x_data_test2.conjugate(), p-q)
      x_data_test = np.concatenate((x_data_test, math_sentence_answer), axis=1)

test_data = np.concatenate((x_data_test.real, x_data_test.imag), axis=1)
target_test_data = np.concatenate((y_data_test.real, y_data_test.imag), axis=1)


print('x데이터')
print(test_data.shape)
print('\n')

print('y데이터')
print(target_test_data.shape)
print('\n')





h[p] =  1 [0.60215819-1.04475382j 0.64357349+1.46131555j 0.23719995+0.48973736j]
h[p] =  1 [-0.93270061-0.95200652j  0.00204506+0.14586515j -0.3596932 -1.30206553j]
h[p] =  3 [ 1.22225972-0.59386705j -0.17724571+0.52796201j  0.8269149 -1.11125805j]
h[p] =  3 [ 0.01324754-0.23742465j -0.00470476-0.48930573j -0.49915285+1.27241448j]
h[p] =  3 [ 0.48062063+2.7490118j   0.7942834 +0.17698001j -1.66376121-0.55912141j]
h[p] =  3 [ 1.27692507-0.82414947j  0.62077586-0.56903377j -0.6100757 -0.36025068j]
h[p] =  5 [-1.57092576-1.18689794j  0.79404758+0.24086549j -0.78705885+0.46118671j]
h[p] =  5 [1.85347253-0.77483545j 0.44650343-0.94811743j 0.7608791 -1.23402265j]
h[p] =  5 [ 0.32344387+0.83597029j -0.13396697+0.45044996j -0.46194311-0.77841157j]
h[p] =  5 [ 0.65805573-0.3542542j  -0.25987531-1.59553717j -1.14493554-0.77957394j]
h[p] =  5 [ 0.21829643+0.08998125j  0.95605482+0.38002584j -2.55312098-0.63627802j]
h[p] =  5 [-0.58426323+0.77874547j -0.63838981-0.82203128j  0.08283267-0.80027563j

In [36]:
tf.model = tf.keras.Sequential()

tf.model.add(tf.keras.layers.Dense(units=2, input_dim=72))  # input_dim=9 gives multi-variable regression
tf.model.add(tf.keras.layers.Activation('linear'))  # this line can be omitted, as linear activation is default

tf.model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))
tf.model.summary()
history = tf.model.fit(train_data, target_data, epochs=100)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 2)                 146       
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
3125/3125 [==============================] - 5s 1ms/step - loss: 4.2008
Epoch 2/100
3125/3125 [==============================] - 4s 1ms/step - loss: 0.3079
Epoch 3/100
3125/3125 [==============================] - 4s 1ms/step - loss: 0.1910
Epoch 4/100
3125/3125 [==============================] - 4s 1ms/step - loss: 0.1421
Epoch 5/100
3125/3125 [==============================] - 4s 1ms/step - loss: 0.1139
Epoch 6/100
3125/3125 [=========

In [44]:
score = tf.model.evaluate(test_data, target_test_data, verbose=1)
print(score) ##  var_N 의 값과 거의 일치한다

3125/3125 [==============================] - 5s 2ms/step - loss: 5.1688e-06
5.1688411986106075e-06
tf.Tensor(5.168846894434848e-06, shape=(), dtype=float64)
